In [1]:
from itertools import combinations

In [76]:
from qnute.helpers.lattice import get_center, manhattan_dist
import numpy as np
from math import isclose

S:set[int] = {3}
u_domains:list[set[int]] = [
    {0,1}, {1,2}, {2,3}, {3,4}
]
invert_map = {i:(i,0) for i in range(5)}
S_center = get_center([invert_map[index] for index in S])
u_domain_centers:list[np.ndarray[float]] = [get_center([invert_map[index] for index in domain]) for domain in u_domains]

print('Support:',S)
r = 1
if len(S) > len(u_domains[0]):
    r+=1
found_flag = False
min_dist = float('inf')
u_index_list = []
while r < len(u_domains) and not found_flag:
    print(f'r={r}')
    for u_indices in combinations(range(len(u_domains)), r):
        u = [u_domains[i] for i in u_indices]
        print('\nCombination:', u)
        x = set.union(*u)
        print('    Union:  ', x)
        if S.issubset(x):
            found_flag = True
            print('    Support is a subset of the union!')
            dist = 0.0
            for i in u_indices:
                dist += manhattan_dist(S_center, u_domain_centers[i])
            print(f'        Distance of cover from support center:    {dist}')
            if isclose(dist,min_dist):
                u_index_list += list(u_indices)
            else:
                if dist < min_dist:
                    min_dist = dist
                    u_index_list = list(u_indices)
            
    print()
    r += 1
if not found_flag:
    u_index_list = list(range(len(u_domains)))
else:
    r -= 1
print(f'Number of covering domains: {r}')
print(f'Minimum Distance: {min_dist}')
print(f'U indices:        {u_index_list}')

Support: {3}
r=1

Combination: [{0, 1}]
    Union:   {0, 1}

Combination: [{1, 2}]
    Union:   {1, 2}

Combination: [{2, 3}]
    Union:   {2, 3}
    Support is a subset of the union!
        Distance of cover from support center:    0.5

Combination: [{3, 4}]
    Union:   {3, 4}
    Support is a subset of the union!
        Distance of cover from support center:    0.5

Number of covering domains: 1
Minimum Distance: 0.5
U indices:        [2, 3]


In [84]:
ham.pterm_list

array([( 1,  1.  +0.j), ( 5,  0.5 +0.j), (10,  0.5 +0.j), (21,  0.25+0.j),
       (26, -0.25+0.j), (38,  0.25+0.j), (41,  0.25+0.j), ( 0, -2.  +0.j)],
      dtype=[('pauli_id', '<u4'), ('amplitude', '<c16')])

In [94]:
def get_minimum_cover(support:set[int], u_domains:list[set[int]], invert_map:dict[int,tuple[int]]) -> list[int]:
    support_center = get_center([invert_map[index] for index in support])
    u_domain_centers:list[np.ndarray[float]] = [get_center([invert_map[index] for index in domain]) for domain in u_domains]

    r = 1
    if len(support) > len(u_domains[0]):
        r+=1
    found_flag = False
    min_dist = float('inf')
    u_index_list = []
    while r < len(u_domains) and not found_flag:
        for u_indices in combinations(range(len(u_domains)), r):
            u = [u_domains[i] for i in u_indices]
            x = set.union(*u)
            if support.issubset(x):
                found_flag = True
                dist = 0.0
                for i in u_indices:
                    dist += manhattan_dist(support_center, u_domain_centers[i])
                if isclose(dist,min_dist):
                    u_index_list += list(u_indices)
                else:
                    if dist < min_dist:
                        min_dist = dist
                        u_index_list = list(u_indices)
                
        # print()
        r += 1
    if not found_flag:
        u_index_list = list(range(len(u_domains)))
    else:
        r -= 1
    return u_index_list

In [95]:
def get_support(ham:Hamiltonian, term:int) -> set[int]:
    p = ham.pterm_list[ham.hm_indices[term]]['pauli_id']
    support = set()
    for i in range(ham.nbits):
        if p % 4 != 0:
            support.add(i)
        p //= 4
    return support

get_support(ham, 2)

{0, 1, 2}

In [98]:
from qnute.hamiltonian import Hamiltonian
from qnute.hamiltonian.laplacian import generateLaplaceHamiltonian1D

num_qbits = 3
ham:Hamiltonian = generateLaplaceHamiltonian1D(num_qbits)
print(ham)
u_domains = [{i,i+1} for i in range(num_qbits-1)]
invert_map = {i:(i,) for i in range(num_qbits)}

amplitude_splits = np.zeros((ham.num_terms,len(u_domains)),dtype=np.float64)

for term in range(ham.num_terms):
    support = get_support(ham, term)
    
    u_index_list = get_minimum_cover(support, u_domains, invert_map)
    frac = 1.0/len(u_index_list)
    for ui in u_index_list:
        amplitude_splits[term,ui] += frac

print(amplitude_splits)

Hamiltonian Pauli Terms and Amplitudes:

	I_2 I_1 X_0  : (1.00000+0.00000j)

	I_2 X_1 X_0  : (0.50000+0.00000j)
	I_2 Y_1 Y_0  : (0.50000+0.00000j)

	X_2 X_1 X_0  : (0.25000+0.00000j)
	X_2 Y_1 Y_0  : (-0.25000+0.00000j)
	Y_2 X_1 Y_0  : (0.25000+0.00000j)
	Y_2 Y_1 X_0  : (0.25000+0.00000j)
	I_2 I_1 I_0  : (-2.00000+0.00000j)

Term 0
Amplitude Splits: [[1. 0.]
 [0. 0.]
 [0. 0.]]
Term 1
Amplitude Splits: [[1. 0.]
 [1. 0.]
 [0. 0.]]
Term 2
Amplitude Splits: [[1.  0. ]
 [1.  0. ]
 [0.5 0.5]]
[[1.  0. ]
 [1.  0. ]
 [0.5 0.5]]


In [101]:
ham.get_hm_pterms(2)

array([(21,  0.25+0.j), (26, -0.25+0.j), (38,  0.25+0.j), (41,  0.25+0.j),
       ( 0, -2.  +0.j)],
      dtype=[('pauli_id', '<u4'), ('amplitude', '<c16')])

In [113]:
from qnute.hamiltonian import hm_dtype
new_pterm_list = np.zeros(len(u_domains)*ham.pterm_list.shape[0], dtype=hm_dtype)
new_hm_indices = []
counter = 0
for i,dom in enumerate(u_domains):
    new_hm_indices.append(counter)
    for term in range(ham.num_terms):
        for pterm in ham.get_hm_pterms(term):
            new_pterm_list[counter]=(pterm['pauli_id'], pterm['amplitude']*amplitude_splits[term,i])
            counter += 1

# new_pterm_list = np.array(new_pterm_list, dtype=hm_dtype)

print(new_pterm_list)
print(new_hm_indices)

[( 1,  1.   +0.j) ( 5,  0.5  +0.j) (10,  0.5  +0.j) (21,  0.125+0.j)
 (26, -0.125+0.j) (38,  0.125+0.j) (41,  0.125+0.j) ( 0, -1.   +0.j)
 ( 1,  0.   +0.j) ( 5,  0.   +0.j) (10,  0.   +0.j) (21,  0.125+0.j)
 (26, -0.125+0.j) (38,  0.125+0.j) (41,  0.125+0.j) ( 0, -1.   +0.j)]
[0, 8]


In [118]:
from copy import deepcopy

ham2 = deepcopy(ham)
ham2.pterm_list = new_pterm_list
ham2.hm_indices = new_hm_indices
ham2.num_terms = len(u_domains)

print(ham2)
print(ham2.get_matrix().real)

Hamiltonian Pauli Terms and Amplitudes:

	I_2 I_1 X_0  : (1.00000+0.00000j)
	I_2 X_1 X_0  : (0.50000+0.00000j)
	I_2 Y_1 Y_0  : (0.50000+0.00000j)
	X_2 X_1 X_0  : (0.12500+0.00000j)
	X_2 Y_1 Y_0  : (-0.12500+0.00000j)
	Y_2 X_1 Y_0  : (0.12500+0.00000j)
	Y_2 Y_1 X_0  : (0.12500+0.00000j)
	I_2 I_1 I_0  : (-1.00000+0.00000j)

	I_2 I_1 X_0  : (0.00000+0.00000j)
	I_2 X_1 X_0  : (0.00000+0.00000j)
	I_2 Y_1 Y_0  : (0.00000+0.00000j)
	X_2 X_1 X_0  : (0.12500+0.00000j)
	X_2 Y_1 Y_0  : (-0.12500+0.00000j)
	Y_2 X_1 Y_0  : (0.12500+0.00000j)
	Y_2 Y_1 X_0  : (0.12500+0.00000j)
	I_2 I_1 I_0  : (-1.00000+0.00000j)

[[-2.  1.  0.  0.  0.  0.  0.  0.]
 [ 1. -2.  1.  0.  0.  0.  0.  0.]
 [ 0.  1. -2.  1.  0.  0.  0.  0.]
 [ 0.  0.  1. -2.  1.  0.  0.  0.]
 [ 0.  0.  0.  1. -2.  1.  0.  0.]
 [ 0.  0.  0.  0.  1. -2.  1.  0.]
 [ 0.  0.  0.  0.  0.  1. -2.  1.]
 [ 0.  0.  0.  0.  0.  0.  1. -2.]]


In [111]:
new_pterm_list

array([( 1,  1.   +0.j), ( 5,  0.5  +0.j), (10,  0.5  +0.j),
       (21,  0.125+0.j), (26, -0.125+0.j), (38,  0.125+0.j),
       (41,  0.125+0.j), ( 0, -1.   +0.j), ( 1,  0.   +0.j),
       ( 5,  0.   +0.j), (10,  0.   +0.j), (21,  0.125+0.j),
       (26, -0.125+0.j), (38,  0.125+0.j), (41,  0.125+0.j),
       ( 0, -1.   +0.j)],
      dtype=[('pauli_id', '<u4'), ('amplitude', '<c16')])